## Finding fusions - Angus's code + my adaptation for soft clips which do NOT have an hg38 map

In [ ]:
#import packages needed packages
import os
import pysam
import re
import pandas as pd

In [ ]:
# Define configurations with detailed Breakpoint_IDs #
# our patient is chr1 not chr4 DUX4 but these are hg38 BAMs so I am jsut putting the corresponding DUX4 coords
patient_info = [
    {
        "PatientID": "patient_FT",
        "BAMs": [
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB14449539/possorted_genome_bam39.bam",
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB14449540/possorted_genome_bam40.bam",
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB14449541/possorted_genome_bam41.bam"
        ],
        "Breakpoints": [
            {"chr1": "19", "pos1": 42295139, "chr2": "4", "pos2": 190174732, "gene1": "CIC", "gene2": "DUX4", "Breakpoint_ID": "CIC_42295139_DUX4_190174732"}
        ]
    },
    {
        "PatientID": "patient_FO",
        "BAMs": [
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960948/possorted_genome_bam48.bam",
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960949/possorted_genome_bam49.bam",
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960950/possorted_genome_bam50.bam",
            "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/possorted_genome_bam51.bam"
        ],
        "Breakpoints": [
            {"chr1": "19", "pos1": 42295139, "chr2": "4", "pos2": 190174732, "gene1": "CIC", "gene2": "DUX4", "Breakpoint_ID": "CIC_42295139_DUX4_190174732"}
        ]
    }
]
# define the output directory #
output_dir = "/lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/"

In [ ]:
# ensure output directory exists
output_dir = "/lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/"
os.makedirs(output_dir, exist_ok=True)

# Define CIGAR regex pattern
cigar_pattern = re.compile(r'(\d+)([MIDNSHP=X])')
wiggle_room = 50  # Allow 50 bp wiggle room around breakpoints

### this is a very big function that pulls out a lot of information about the breakpoints and soft clipped reads and their status
# it also handles the soft clipped portions of the sequences
# I have adapted this approach from Angus Hodder, a PhD student from the lab
def extract_read_metrics(read, bam_prefix):
    read_status = "Unmapped" if read.is_unmapped else ("Multi-mapped" if read.has_tag("NH") and read.get_tag("NH") > 1 else "Mapped")
    num_alignments = read.get_tag("NH") if read.has_tag("NH") else None
    alignment_score = read.get_tag("AS") if read.has_tag("AS") else None
    read_sequence = read.query_sequence if read.query_sequence else "No sequence"

    ## the function also extracts the barcodes where the soft clipped reads are present
    cell_barcode = dict(read.get_tags()).get("CB") or dict(read.get_tags()).get("CR")
    cell_barcode = cell_barcode.replace("-1-1", "-1") if cell_barcode else "Unknown"
    cell_id = f"{bam_prefix}::{cell_barcode}" if cell_barcode != "Unknown" else "Unknown"

    # here it handles soft clipped portions and determines last mapped position
    cigar_string = read.cigarstring if read.cigarstring else ""
    soft_clipped_start = ""
    soft_clipped_end = ""
    start_position = read.reference_start
    end_position = read.reference_end

    # initialize positions for soft clips
    soft_clip_start_pos = None
    soft_clip_end_pos = None

    # parse the cigar strings for soft clipped portions 
    parsed_cigar = cigar_pattern.findall(cigar_string)
    current_pos = read.reference_start

    for length, operation in parsed_cigar:
        length = int(length)
        if operation == "S":
            if current_pos == read.reference_start:
                # soft-clipped at the start of read
                soft_clipped_start = read_sequence[:length]
                soft_clip_start_pos = current_pos - length  # position before the start of the alignment
            else:
                # Ssft-clipped at the end of read
                soft_clipped_end = read_sequence[-length:]
                soft_clip_end_pos = current_pos  # position at the end of the alignment
        elif operation in ("M", "D", "N", "="):
            current_pos += length

    # update last mapped position if the read ends with soft-clipping
    if soft_clipped_end:
        end_position = current_pos - 1

    # full read sequence including soft clipped portions
    full_read_sequence = f"{soft_clipped_start}{read_sequence}{soft_clipped_end}"

    # here also return additional info with soft clip positions - this is very useful for assigning barcodes in retrospect
    return read_status, num_alignments, alignment_score, full_read_sequence, cell_barcode, cell_id, end_position, soft_clipped_start, soft_clipped_end, soft_clip_start_pos, soft_clip_end_pos

# Main processing loop 
for patient in patient_info: # I have only one patient which I earlier named patient_FO and patient_FT
    patient_id = patient["PatientID"]
    breakpoints = patient["Breakpoints"]

    for bam_path in patient["BAMs"]: 
        bam_prefix = os.path.basename(os.path.dirname(bam_path))
        output_file = os.path.join(output_dir, f"{patient_id}_{bam_prefix}_reads_over_breakpoints.tsv")
        output_data = []

        # here I am actually extracting info from the bam file using pysam on the chromosome breaks
        with pysam.AlignmentFile(bam_path, "rb") as bam:
            for bp in breakpoints:
                chr1, pos1, chr2, pos2 = bp["chr1"], bp["pos1"], bp["chr2"], bp["pos2"]
                gene1, gene2 = bp["gene1"], bp["gene2"]

                # define the Breakpoint_ID
                breakpoint_id = f"{gene1}_{pos1}_{gene2}_{pos2}"

                # fetch reads overlapping the breakpoints
                for read in bam.fetch(chr1, pos1 - wiggle_room, pos1 + wiggle_room):
                    read_status, num_alignments, alignment_score, full_read_sequence, cell_barcode, cell_id, last_mapped_position, soft_clipped_start, soft_clipped_end, soft_clip_start_pos, soft_clip_end_pos = extract_read_metrics(read, bam_prefix)
                    output_data.append([ 
                        patient_id, bam_path, read.query_name, chr1, read.reference_start,
                        read.reference_end, read_status, num_alignments, alignment_score,
                        full_read_sequence, cell_barcode, cell_id, breakpoint_id,
                        last_mapped_position, soft_clipped_start, soft_clipped_end, soft_clip_start_pos, soft_clip_end_pos
                    ])

                ## define some space for wiggle
                for read in bam.fetch(chr2, pos2 - wiggle_room, pos2 + wiggle_room):
                    read_status, num_alignments, alignment_score, full_read_sequence, cell_barcode, cell_id, last_mapped_position, soft_clipped_start, soft_clipped_end, soft_clip_start_pos, soft_clip_end_pos = extract_read_metrics(read, bam_prefix)
                    output_data.append([ 
                        patient_id, bam_path, read.query_name, chr2, read.reference_start,
                        read.reference_end, read_status, num_alignments, alignment_score,
                        full_read_sequence, cell_barcode, cell_id, breakpoint_id,
                        last_mapped_position, soft_clipped_start, soft_clipped_end, soft_clip_start_pos, soft_clip_end_pos
                    ])

        # define columns and save the extracted data
        columns = [
            "PatientID", "BAM", "ReadName", "Chromosome", "Start", "End",
            "ReadStatus", "NumAlignments", "AlignmentScore", "FullReadSequence",
            "CellBarcode", "CellID", "Breakpoint_ID", "LastMappedPosition",
            "SoftClippedStart", "SoftClippedEnd", "SoftClipStartPos", "SoftClipEndPos"
        ]
        # save into dataframes - tsv
        df = pd.DataFrame(output_data, columns=columns)
        df.to_csv(output_file, sep='\t', index=False)
        print(f"Completed processing for patient {patient_id}. Output saved to: {output_file}")

print("All patient data processed successfully.")

Completed processing for patient patient_FT. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/patient_FT_CG_SB_NB14449539_reads_over_breakpoints.tsv
Completed processing for patient patient_FT. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/patient_FT_CG_SB_NB14449540_reads_over_breakpoints.tsv
Completed processing for patient patient_FT. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/patient_FT_CG_SB_NB14449541_reads_over_breakpoints.tsv
Completed processing for patient patient_FO. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/patient_FO_CG_SB_NB13960948_reads_over_breakpoints.tsv
Completed processing for patient patient_FO. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_outputs/patient_FO_CG_SB_NB13960949_reads_over_breakpoints.tsv
Completed processing for patient patient_FO. Output saved to: /lustre/scratch126/casm/team274sb/lr26/scRNA/fusion_o

In [ ]:
### for my purpose with the breakpoint leading to an undefined location this only worked for the sake of finding the soft clips 
# I had to adapt the approach further to just extract the soft clips from the exon 21 CIC and then blat them, retaining cell barcodes for each 
# packages needed
import pysam
import re
import os
import csv
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

# here I am defining a similar function to Angus's and I want at least 20 bp long
def extract_soft_clipped_reads_with_barcodes(bamfile_path, chrom, start, end, min_softclip=20, bam_prefix="BAM", patient_id="Sample"):
    soft_clipped_reads = []
    cigar_pattern = re.compile(r'(\d+)([MIDNSHP=X])')
    # open the bam file to find reads that match 
    with pysam.AlignmentFile(bamfile_path, "rb") as bamfile:
        for read in bamfile.fetch(chrom, start, end):
            if read.is_unmapped:
                continue

            # retain the cell barcode and ID and if none then put unknown 
            cell_barcode = dict(read.get_tags()).get("CB") or dict(read.get_tags()).get("CR")
            cell_barcode = cell_barcode.replace("-1-1", "-1") if cell_barcode else "Unknown"
            cell_id = f"{bam_prefix}::{cell_barcode}" if cell_barcode != "Unknown" else "Unknown"
            # obtain strand orientation, cigar read info and the actual read sequence
            strand = "-" if read.is_reverse else "+"
            cigar_tuples = read.cigartuples
            cigar_string = read.cigarstring or ""
            read_sequence = read.query_sequence

            if not cigar_tuples or not read_sequence:
                continue

            # determine if soft clipping exists and meets threshold
            # I initiate similarly to in Angus's approach
            soft_clipped_seq = None
            soft_clipped_start = ""
            soft_clipped_end = ""
            soft_clip_start_pos = None
            soft_clip_end_pos = None
            # and parse cigars in a very similar way
            parsed_cigar = cigar_pattern.findall(cigar_string)
            current_pos = read.reference_start
            for i, (length, op) in enumerate(parsed_cigar):
                length = int(length)
                if op == "S":
                    if i == 0 and length >= min_softclip:
                        soft_clipped_start = read_sequence[:length]
                        soft_clip_start_pos = current_pos - length
                    elif i == len(parsed_cigar) - 1 and length >= min_softclip:
                        soft_clipped_end = read_sequence[-length:]
                        soft_clip_end_pos = current_pos
                elif op in ("M", "D", "N", "="):
                    current_pos += length
            # here I am obtaining the info that I store including the sample (patientID here), read ID, cell ID and cell barcode, from which bam it is etc
            if soft_clipped_start or soft_clipped_end:
                aligned_start = read.reference_start
                aligned_end = read.reference_end
                soft_clipped_reads.append({
                    "PatientID": patient_id,
                    "BAM": bamfile_path,
                    "ReadID": read.query_name,
                    "Chromosome": chrom,
                    "Start": aligned_start,
                    "End": aligned_end,
                    "Strand": strand,
                    "CellBarcode": cell_barcode,
                    "CellID": cell_id,
                    "SoftClippedStart": soft_clipped_start,
                    "SoftClippedEnd": soft_clipped_end,
                    "SoftClipStartPos": soft_clip_start_pos,
                    "SoftClipEndPos": soft_clip_end_pos
                })

    return soft_clipped_reads
# here I just define the column names and write into the csv file
def save_soft_clipped_reads_to_csv(soft_clipped_reads, output_path):
    fieldnames = [
        "PatientID", "BAM", "ReadID", "Chromosome", "Start", "End", "Strand",
        "CellBarcode", "CellID", "SoftClippedStart", "SoftClippedEnd", 
        "SoftClipStartPos", "SoftClipEndPos"
    ]
    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in soft_clipped_reads:
            writer.writerow(row)
# and here I read the soft clipped reads, their starts, ends, read ID
### IMPORTANT note: CG_SB_NB1444954 is not dynamic and represents the channel run, I can afford to change this manually 
# as I have very few channels but should be dynamic for bigger datasets
# then write into a fasta file
def save_soft_clips_to_fasta(soft_clipped_reads, fasta_path):
    records = []
    for i, read in enumerate(soft_clipped_reads):
        for side, seq in [('start', read['SoftClippedStart']), ('end', read['SoftClippedEnd'])]:
            if seq:
                record_id = f"{read['ReadID']}|CG_SB_NB1444954:{read['CellBarcode']}|{side}"
                records.append(SeqRecord(Seq(seq), id=record_id, description=""))
    SeqIO.write(records, fasta_path, "fasta")

### IMPORTANT this is the main function. It is non-dynamic. It has hard-embedded input which I change depending on where my sample is and where I want my output to be and my fasta ouput to be.
### I have used this function for all my seven channels, changing dynamically the bamfile_path, output_csv and fasta_output.
if __name__ == "__main__":
    bamfile_path = "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/possorted_genome_bam51.bam"
    chromosome = "19"
    start = 42295000
    end = 42295200
    output_csv = "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/soft_clipped_reads_chr19.csv"

    results = extract_soft_clipped_reads_with_barcodes(bamfile_path, chromosome, start, end, min_softclip=10, bam_prefix="Run1", patient_id="P01")
    save_soft_clipped_reads_to_csv(results, output_csv)
    print(f"Extracted {len(results)} soft-clipped reads. Output saved to {output_csv}.")
    
    fasta_output = "/lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/soft_clipped_reads_chr19.fasta"
    save_soft_clips_to_fasta(results, fasta_output)
    print(f"FASTA file saved to {fasta_output}.")


Extracted 9 soft-clipped reads. Output saved to /lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/soft_clipped_reads_chr19.csv.
FASTA file saved to /lustre/scratch126/casm/team274sb/lr26/scRNA/CG_SB_NB13960951/soft_clipped_reads_chr19.fasta.


In [ ]:
### AFTER EXTRACTING FASTA, use the blat-sc.sh script which will handle finding of the soft-clipped sequences against the T2T reference. ###
### I have then used all the positive sequence cell barcodes leading to chr1 D4Z4 breakpoint (since it is shallow snRNA data there were not many) and defined them in a list as input for UMAP.
### I saved the fastas locally, I perfromed the blat-sc.sh in the new filesystem /cellgen/behjati/.